In [122]:
#import torch
from torch_geometric.loader import DataLoader
import numpy as np
import pandas as pd
import os
from struc_feat import ego_label_set, compute_contacts  
import os
from torch_geometric.data import Data
import re
import ast
from multiprocessing import Pool
from joblib import Parallel, delayed
from Bio.PDB import PDBParser
import networkx as nx
import torch

In [138]:
csv = pd.read_csv('/Users/robsonlab/Teetly/get_data_pyrosetta/non_enz.csv')

In [139]:
csv

,Entry,EC number,EC_Shortened,Active site,Binding site,Reviewed,Entry Name,Protein names,Gene Names,Organism,...,pH dependence,Pathway,Kinetics,Activity regulation,Redox potential,Rhea ID,Site,Temperature dependence,PIRSF,PDB
0,A0A061ACU2,[],[],[],[],reviewed,PIEZ1_CAEEL,Piezo-type mechanosensitive ion channel compon...,pezo-1 C10C5.1,Caenorhabditis elegans,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4PKE;4PKX;
1,A0A068B6Q6,[],[],[],[],reviewed,CA18_CONBE,Conotoxin Bt1.8,NaN,Conus betulinus (Beech cone),...,NaN,NaN,NaN,NaN,NaN,NaN,"SITE 29; /note=""Important for binding to nAChR...",NaN,NaN,2NAY;
2,A0A097PTA8,[],[],[],[],reviewed,DEFCO_COPCI,Fungal defensin copsin,NaN,Coprinopsis cinerea (Inky cap fungus) (Hormogr...,...,BIOPHYSICOCHEMICAL PROPERTIES: pH dependence:...,NaN,NaN,NaN,NaN,NaN,NaN,BIOPHYSICOCHEMICAL PROPERTIES: Temperature de...,NaN,2MN5;
3,A0A0D1DWZ5,[],[],[],[],reviewed,RRM4_USTMA,RNA-binding protein RRM4,RRM4 UMAG_10836,Ustilago maydis (strain 521 / FGSC 9021) (Corn...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7PZE;
4,A0A0D2UG83,[],[],[],"[225, 226, 203, 204, 275, 313, 314, 315, 316, ...",reviewed,H2AY_CAPO3,Histone macroH2A1.1,CAOG_004778,Capsaspora owczarzaki (strain ATCC 30864),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7NY6;7NY7;
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4886,Q8YT41,[],[],[],[],reviewed,KAIB_NOSS1,Circadian clock oscillator protein KaiB,kaiB alr2885,Nostoc sp. (strain PCC 7120 / SAG 25.82 / UTEX...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1R5P;
4887,Q8YT42,[],[],[],[],reviewed,KAIA_NOSS1,Circadian clock oscillator protein KaiA,kaiA alr2884,Nostoc sp. (strain PCC 7120 / SAG 25.82 / UTEX...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1R5Q;
4888,Q97Y91,[],[],[],[],reviewed,CSA2B_SACS2,CRISPR-associated aCascade subunit Cas7/Csa2 2...,csa2b cas7b SSO1442,Saccharolobus solfataricus (strain ATCC 35092 ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3PS0;
4889,Q97YC8,[],[],[],[],reviewed,CSA5A_SACS2,CRISPR type I-A cluster 1/Apern-associated pro...,csa5 SSO1398,Saccharolobus solfataricus (strain ATCC 35092 ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3ZC4;


In [49]:
row = csv.iloc[1]


In [50]:
id = row['Entry']
file_list = list(os.listdir("/Users/robsonlab/Teetly/wildtype_pdbs"))
r = re.compile(f'{id}*')

file_name = list(filter(r.match, file_list))[0]

In [51]:
active_binding_sites = ast.literal_eval(row['Active site'])
active_binding_sites.extend(ast.literal_eval(row['Binding site']))
active_binding_sites.sort()

start_site = int(file_name.split('_')[2])
end_site = int(file_name.split('_')[3].split('.')[0])
#clamp to what's in that range
active_binding_sites = [site for site in active_binding_sites if site >= start_site and site <= end_site]

In [52]:
#parse pdb
pdb_path = f'/Users/robsonlab/Teetly/wildtype_pdbs/{file_name}'
parser = PDBParser()
protein = parser.get_structure(id, pdb_path)
protein = protein[0]

In [53]:
#print residue sequence 
residues = list(protein.get_residues())
assert len(residues) == end_site - start_site + 1

In [54]:
from struc_feat import load_pdb

In [55]:
node_labels, coords, lrfs, residue_one_hot = load_pdb(protein)

In [56]:
residue_one_hot = torch.tensor(residue_one_hot, dtype = torch.float)

In [57]:
pos = torch.tensor(coords, dtype=torch.float)

In [58]:
angle_geom = torch.tensor(lrfs, dtype=torch.float)

In [59]:
def create_protein_graph(active_and_binding_site_residues, protein, start_site):
    """For a given protein pdb_id, extract all functional site annotations and create a graph where the contact map is
    the adjancency matrix, nodes are labelled according to "chain_position_residue" and functionality of nodes is depicted.

    Args:
        PDB_ID (str): PDB ID of the protein which is to be converted to a graph
        df (pd.DataFrame): Dataframe containing the PDBSite annotations

    Returns:
        protein_graph (nx.Graph): graph with attributes such as functionality, pdb_site_id, length of functional site,
                                edges are the contacts between residues
    """
    
    node_labels, coords, lrfs, residue_one_hot = load_pdb( protein)
    contacts = compute_contacts(coords, node_labels)
    # plt.imshow(contacts)
    # print(node_labels, info_dict, coords)
    assert contacts.shape[0] == len(node_labels)
    print(len(node_labels))
    protein_graph = nx.from_numpy_matrix(contacts)
    
    # protein_graph.edges
    nx.set_node_attributes(
        protein_graph, name="y", values=0
    )  # 0 for non-functional, 1 for functional
    ##### groupby protein structure, set all nodes to none
    ##### select functional sites, put label function, subtype: header
    #take in the active and binding site residues and label their respective nodes as y = 1
    for res_num in active_and_binding_site_residues:
        #index for that residue number
        try:
            protein_graph.nodes[res_num-start_site]["y"] = 1
        except:
            pass
    return protein_graph

In [61]:
graph  = create_protein_graph(active_binding_sites, protein, start_site)

590


In [62]:
y =  torch.tensor([att["y"] for node, att in graph.nodes(data=True)], dtype=torch.float)

In [69]:
label_graphs = ego_label_set(graph, active_binding_sites, start_site)

torch.Size([590])


In [70]:
label_graphs

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 

In [71]:
edge_index = torch.LongTensor(list(graph.edges)).t().contiguous()

In [68]:
def ego_label_set(graph: nx.Graph, sites: list, start_site:int, 
                  radius = 3, 
                  overlap_ratio_cutoff = 0.7):
    label_graphs = torch.zeros(len(graph.nodes), dtype=torch.float)
    print(label_graphs.shape)
    functional_nodes = [
        node for node, att in graph.nodes(data=True) if att["y"] == 1
    ]
    for functional_node in functional_nodes:
        ego_subgraph = nx.ego_graph(graph, functional_node, radius=radius)
        #count number of functional nodes in ego subgraph
        func_subgraph_nodes = len([
            node for node, att in ego_subgraph.nodes(data=True) if att["y"] == 1
        ]) + 1
        #now find number of functional nodes within 10 angstroms of functional node
        #extract site nodes numbers, and compute distances
        total_func_nodes_ten_apart = 1  
        for site in sites:
            try:
                if nx.shortest_path_length(graph, source=functional_node, target=site, weight='distance') <= 10: #in try except since nodes may not be reachable, but there is no shortest path thing that coudl return null
                    total_func_nodes_ten_apart += 1
            except:
                pass
        #if this functional node has at least [overlap_ratio_cutoff] of the functional nodes within 10 angstroms of it, give it an ego label of 1 and add it
        #to label_graphs dictionary
        
        if func_subgraph_nodes / total_func_nodes_ten_apart >= overlap_ratio_cutoff:
            label_graphs[functional_node] = 1
        
    # nx.set_node_attributes(graph, ego_label, "ego_label")
    return label_graphs

In [77]:
ecs = ast.literal_eval(row['EC_Shortened'])
#len(label_graphs)

In [86]:
ec_vals = torch.zeros(8, dtype=torch.float)
if len(ecs) > 0:
    for ec in ecs:
        ec_vals[ec] = 1

Inspections

In [140]:
data = torch.load("/Users/robsonlab/Teetly/tmp/A0A0G2KTI4_6NPH_206_677.pdb.pt")

In [118]:
data

tensor([[1]])

In [142]:
#sum data.x along x
data.y.nonzero()

tensor([[ 13],
        [ 14],
        [ 15],
        [ 16],
        [ 17],
        [ 18],
        [ 19],
        [ 88],
        [ 99],
        [211],
        [212],
        [214],
        [215],
        [216],
        [329],
        [332],
        [333],
        [401],
        [405]])

In [145]:
#sum total 1s in data.y
data.label_graphs.nonzero()

tensor([[ 13],
        [ 14],
        [ 15],
        [ 16],
        [ 17],
        [ 18],
        [ 19],
        [ 99],
        [211],
        [212],
        [214],
        [215],
        [216],
        [329],
        [332],
        [333]])

In [141]:
data.ec_number

tensor([0., 0., 0., 0., 0., 0., 0., 0.])

In [111]:
#sum total ones in data.label_graphs
data.ec_number

tensor([0., 0., 0., 0., 1., 0., 0., 0.])